##### Copyright 2024 Google LLC.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Prompting with media files

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/gemini-api/docs/prompting_with_media"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />View on ai.google.dev</a>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google/generative-ai-docs/blob/main/site/en/gemini-api/docs/prompting_with_media.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google/generative-ai-docs/blob/main/site/en/gemini-api/docs/prompting_with_media.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

The Gemini API supports *multimodal* prompting with text, image, and audio data. For small files, you can point the Gemini model
directly to a local file when providing a prompt. Upload larger files with the
[File API](https://ai.google.dev/api/rest/v1beta/files) before including them in
prompts.

The File API lets you store up to 20GB of files per project, with each file not
exceeding 2GB in size. Files are stored for 48 hours and can be accessed with
your API key for generation within that time period and cannot be downloaded from the API. It is available at no cost in all regions where the [Gemini API is
available](https://ai.google.dev/available_regions).

The File API handles inputs that can be used to generate content with [`model.generateContent`](https://ai.google.dev/api/rest/v1/models/generateContent) or [`model.streamGenerateContent`](https://ai.google.dev/api/rest/v1/models/streamGenerateContent). For information on valid file formats (MIME types) and supported models, see [Supported file formats](#supported_file_formats).

This guide shows how to use the File API to upload media files and include them in a `GenerateContent` call to the Gemini API. For more information, see the [code
samples](https://github.com/google-gemini/gemini-api-cookbook/tree/main/quickstarts/file-api).


## Setup

Before you use the File API, you need to install the Gemini API SDK package and configure an API key. This section describes how to complete these setup steps.

### Install the Python SDK and import packages

The Python SDK for the Gemini API is contained in the [google-generativeai](https://pypi.org/project/google-generativeai/) package. Install the dependency using pip.

In [1]:
!pip install -q -U google-generativeai

Import the necessary packages.

In [4]:
import google.generativeai as genai
from IPython.display import Markdown

### Setup your API key

The File API uses API keys for authentication and access. Uploaded files are associated with the project linked to the API key. Unlike other Gemini APIs that use API keys, your API key also grants access to data you've uploaded to the File API, so take extra care in keeping your API key secure. For more on keeping your keys
secure, see [Best practices for using API
keys](https://support.google.com/googleapi/answer/6310037).

Store your API key in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or are unfamiliar with Colab Secrets, refer to the [Authentication quickstart](https://github.com/google-gemini/gemini-api-cookbook/blob/main/quickstarts/Authentication.ipynb).

In [5]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

## Prompting with images


### Upload an image file to the File API

In this tutorial, you upload a sample image to the API and use it to generate content.

Refer to the [Appendix section](#uploading_files_to_colab) to learn how to upload your own file.

In [ ]:
!curl -o image.jpg https://storage.googleapis.com/generativeai-downloads/images/jetpack.jpg

In [ ]:
# Upload the file
sample_file = genai.upload_file(path="image.jpg",
                            display_name="Sample drawing")

print(f"Uploaded file '{sample_file.display_name}' as: {sample_file.uri}")

The `response` shows that the File API stored the specified `display_name` for the uploaded file and a `uri` to reference the file in Gemini API calls. Use `response` to track how uploaded files are mapped to URIs.

Depending on your use case, you can also store the URIs in structures such as a `dict` or a database.

### Get file

After uploading the file, verify that the File API has successfully received it by calling `files.get`.

`files.get` lets you get the file metadata that have been uploaded to the File API that are associated with the Cloud project your API key belongs to. Only the `name` (and by extension, the `uri`) are unique. Only use the `displayName` to identify files if you manage uniqueness yourself.

In [ ]:
file = genai.get_file(name=sample_file.name)
print(f"Retrieved file '{file.display_name}' as: {sample_file.uri}")

### Generate content

After uploading the file, you can make `GenerateContent` requests that reference the File API URI. In this example, you create a prompt that starts with a text followed by the uploaded image.

In [ ]:
# Set the model to Gemini 1.5 Pro.
model = genai.GenerativeModel(model_name="models/gemini-1.5-pro-latest")

response = model.generate_content(["Describe the image with a creative description.", sample_file])

Markdown(">" + response.text)

### Delete files

Files are automatically deleted after 2 days. You can also manually delete them using `files.delete()`.

In [ ]:
genai.delete_file(sample_file.name)
print(f'Deleted {sample_file.display_name}.')

## Prompting with videos

### Upload a video file to the File API

The File API accepts video file formats directly. This example uses the short film "Big Buck Bunny".

> "Big Buck Bunny" is (c) copyright 2008, Blender Foundation / www.bigbuckbunny.org and [licensed](https://peach.blender.org/about/) under the [Creative Commons Attribution 3.0](http://creativecommons.org/licenses/by/3.0/) License.

Refer to the [Appendix section](#uploading_files_to_colab) to learn how to upload your own file.

In [ ]:
!wget https://download.blender.org/peach/bigbuckbunny_movies/BigBuckBunny_320x180.mp4

In [ ]:
my_filename = "your_file_name.mp4"
my_file_display_name = "Your Desired Display Name"

my_file = genai.upload_file(path=my_filename,
                            display_name=my_file_display_name)
print(f"Uploaded file '{my_file.display_name}' as: {my_file.uri}")

In [1]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving youtube video chat.mp4 to youtube video chat.mp4
User uploaded file "youtube video chat.mp4" with length 145191740 bytes


In [6]:
video_file_name = "youtube video chat.mp4"

print(f"Uploading file...")
video_file = genai.upload_file(path=video_file_name)
print(f"Completed upload: {video_file.uri}")

Uploading file...
Completed upload: https://generativelanguage.googleapis.com/v1beta/files/z93jgwx7h77a


### Get file

Verify the API has successfully received the files by calling the `files.get` method.

NOTE: Video files have a `State` field in the File API. When a video is uploaded, it will be in `PROCESSING` state until it is ready for inference. Only `ACTIVE` files can be used for model inference.

In [10]:
import time

while video_file.state.name == "PROCESSING":
    print('.', end='')
    time.sleep(10)
    video_file = genai.get_file(video_file.name)

if video_file.state.name == "FAILED":
  raise ValueError(video_file.state.name)

### Generate content

After the video has been uploaded, you can make `GenerateContent` requests that reference the File API URI.

In [11]:
# Create the prompt.
prompt = "Describe this video."

# Set the model to Gemini 1.5 Pro.
model = genai.GenerativeModel(model_name="models/gemini-1.5-pro-latest")

# Make the LLM request.
print("Making LLM inference request...")
response = model.generate_content([prompt, video_file],
                                  request_options={"timeout": 600})
print(response.text)

Making LLM inference request...
The screen recording shows the use of a YouTube transcription and AI chatbot application called Streamlit. The left half of the screen shows the application, and the right half of the screen shows the YouTube page from which a video is being accessed.

The YouTube video, “Building AI-Powered Chatbots for Life Sciences Applications With AWS,” is pasted into the “YouTube URL” bar, then the “Submit” button is clicked. The video is processed and appears in the “Processed videos” section. In the bar at the bottom, the user types, “explain Dataiku AI chatbots for life sciences Applications,” and clicks the play button to the right. Information on Dataiku AI chatbots for life sciences Applications appears in bullet form at the top of the left half of the screen. The information provided includes links to other YouTube videos. The user then searches for these videos using the app, and watches the videos.


### Delete files

Files are automatically deleted after 2 days or you can manually delete them using `files.delete()`.

In [ ]:
genai.delete_file(video_file.name)
print(f'Deleted file {video_file.uri}')

## Supported file formats

Gemini models support prompting with multiple file formats. This section explains considerations in using general media formats for prompting, specifically image, audio, video, and plain text files. You can use media files for prompting only with specific model versions, as shown in the following table.

<table>
  <thead>
    <tr>
      <th><strong>Model</strong></th>
      <th><strong>Images</strong></th>
      <th><strong>Audio</strong></th>
      <th><strong>Video</strong></th>
      <th><strong>Plain text</strong></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>Gemini 1.5 Pro (release 008 and later)</td>
      <td>✔ (3600 max image files)</td>
      <td>✔</td>
      <td>✔</td>
      <td>✔</td>
    </tr>
    <tr>
      <td>Gemini Pro Vision</td>
      <td>✔ (16 max image files)</td>
      <td></td>
      <td></td>
      <td>✔</td>
    </tr>
  </tbody>
</table>

### Image formats

You can use image data for prompting Gemini models. When you use images for prompting, they are subject to the following limitations and requirements:

-   Images must be in one of the following image data [MIME types](https://developers.google.com/drive/api/guides/ref-export-formats):
    -   PNG - image/png
    -   JPEG - image/jpeg
    -   WEBP - image/webp
    -   HEIC - image/heic
    -   HEIF - image/heif
-   Maximum of 3600 images for `gemini-1.5-pro`
-   No specific limits to the number of pixels in an image; however, larger images are scaled down to fit a maximum resolution of 3072 x 3072 while preserving their original aspect ratio.

### Audio formats

You can use audio data for prompting with the `gemini-1.5-pro` model. When you use audio for prompting, they are subject to the following limitations and requirements:

-   Audio data is supported in the following common audio format [MIME types](https://developers.google.com/drive/api/guides/ref-export-formats):
    -   WAV - audio/wav
    -   MP3 - audio/mp3
    -   AIFF - audio/aiff
    -   AAC - audio/aac
    -   OGG Vorbis - audio/ogg
    -   FLAC - audio/flac
-   The maximum supported length of audio data in a single prompt is 9.5 hours.
-   Audio files are resampled down to a 16 Kbps data resolution, and multiple channels of audio are combined into a single channel.
-   There is no specific limit to the number of audio files in a single prompt, however the total combined length of all audio files in a single prompt cannot exceed 9.5 hours.

### Video formats

You can use video data for prompting with the `gemini-1.5-pro` model.

- Video data is supported in the following common video format [MIME types](https://developers.google.com/drive/api/guides/ref-export-formats):
  -   video/mp4
  -   video/mpeg
  -   video/mov
  -   video/avi
  -   video/x-flv
  -   video/mpg
  -   video/webm
  -   video/wmv
  -   video/3gpp

- The File API service currently samples videos into images at 1 frame per second (FPS) and may be subject to change to provide the best inference quality.
- Video limits depend on the context size limit of the model used for inference. For example, `gemini-1.5-pro` has a maximum video length of ~60 minutes.

### Plain text formats

The File API supports uploading plain text files with the following [MIME types](https://developers.google.com/drive/api/guides/ref-export-formats):
-   text/plain
-   text/html
-   text/css
-   text/javascript
-   application/x-javascript
-   text/x-typescript
-   application/x-typescript
-   text/csv
-   text/markdown
-   text/x-python
-   application/x-python-code
-   application/json
-   text/xml
-   application/rtf
-   text/rtf

For plain text files with a MIME type not on the list, you can try specifying
one of the above MIME types manually.

## Appendix: Uploading files to Colab
<a name="uploading_files_to_colab"></a>
This notebook uses the File API with files that were downloaded from the internet. If you're running this in Colab and want to use your own files, you first need to upload them to the colab instance.

First, click **Files** on the left sidebar, then click the **Upload** button:

<img width=400 src="https://ai.google.dev/tutorials/images/colab_upload.png">

Next, you'll upload that file to the File API. In the form for the code cell below, enter the filename for the file you uploaded and provide an appropriate display name for the file, then run the cell.


In [ ]:
my_filename = "image.jpg" # @param {type:"string"}
my_file_display_name = "Sample Image" # @param {type:"string"}

my_file = genai.upload_file(path=my_filename,
                            display_name=my_file_display_name)
print(f"Uploaded file '{my_file.display_name}' as: {my_file.uri}")